# Importing libraries

In [91]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
 
from numpy import log,dot,exp,shape

from sklearn.impute import KNNImputer 

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense


# Reading dataset

In [92]:
df = pd.read_csv('onehr-data.csv')
df_original = pd.read_csv('onehr-data.csv')

# Infromation about dataset including size, and statistical infromation

In [93]:
print(df.shape)
df.describe()

(2536, 74)


,result
count,2536.000000
mean,0.028785
std,0.167236
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


# Show dataset

In [94]:
df.head()

,Date,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
0,1/1/1998,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,1/2/1998,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,1/3/1998,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,1/4/1998,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,1/5/1998,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,...,?,?,?,?,?,?,?,?,0.58,0.0


# Remove Date column(It isn't useful for prediction)

In [95]:
df.drop(columns='Date', inplace=True)
df.head()

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
0,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,2.3,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,?,?,?,?,?,?,?,?,0.58,0.0


# Handling missing values

In [96]:
df.replace(to_replace='?', value=np.nan, inplace=True)
df.head()

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
0,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,2.3,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.58,0.0


In [97]:
# Splitting the dataset into feature(X) and target(Y)
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [98]:
imputer = KNNImputer(n_neighbors=3, weights="distance")
#imputer = SimpleImputer(missing_values=np.nan, strategy='median')
#df.dropna(inplace=True,axis=0)
#df.interpolate(method='slinear',limit_direction ='both')

In [99]:
df = pd.DataFrame(imputer.fit_transform(df), dtype='float64')
df.columns = ['WSR0','WSR1','WSR2','WSR3','WSR4','WSR5','WSR6','WSR7','WSR8','WSR9','WSR10','WSR11','WSR12','WSR13','WSR14','WSR15','WSR16','WSR17','WSR18','WSR19','WSR20','WSR21','WSR22','WSR23','WSR_PK','WSR_AV','T0','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20','T21','T22','T23','T_PK','T_AV','T85','RH85','U85','V85','HT85','T70','RH70','U70','V70','HT70','T50','RH50','U50','V50','HT50','KI','TT','SLP','SLP_','Precp','Result']
df.head()

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,Result
0,0.8,1.8,2.4,2.1,2.0,2.1,1.5,1.7,1.9,2.3,...,0.15,10.67,-1.56,5795.0,-12.10,17.90,10330.0,-55.0,0.00,0.0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.48,8.39,3.84,5805.0,14.05,29.00,10275.0,-55.0,0.00,0.0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.60,6.94,9.80,5790.0,17.90,41.30,10235.0,-40.0,0.00,0.0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.49,8.73,10.54,5775.0,31.15,51.70,10195.0,-40.0,2.08,0.0
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,0.61,-9.18,-0.87,5905.0,32.55,45.65,10175.0,15.0,0.58,0.0


In [100]:
# check if NaN values exist
if np.nan in df['T_PK'].values.tolist():
  print("NaN values found")
else:
  print("No NaN values found")

No NaN values found


In [101]:
X = imputer.fit_transform(x)

# Kfold cross validation

In [102]:
kf = KFold(n_splits=3, shuffle=True, random_state=1)

# Logistic regression with inbuilt functions of python

In [103]:
def standardize(X_tr):
    for i in range(X_tr.shape[1]):
        X_tr[:,i] = (X_tr[:,i] - np.mean(X_tr[:,i]))/np.std(X_tr[:,i])
def evaluation(y,y_hat):
    tp,tn,fp,fn = 0,0,0,0
    for i in range(len(y)):
        if y[i] == 1 and y_hat[i] == 1:
            tp += 1
        elif y[i] == 1 and y_hat[i] == 0:
            fn += 1
        elif y[i] == 0 and y_hat[i] == 1:
            fp += 1
        elif y[i] == 0 and y_hat[i] == 0:
            tn += 1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy = (tp+tn)/(tp+tn+fn+fp)
    f1_score = 2*precision*recall/(precision+recall)
    return precision,accuracy
class my_LogisticRegression() :
    def __init__( self, learning_rate, iterations ) :        
        self.learning_rate = learning_rate        
        self.iterations = iterations
    # Function for model training    
    def fit( self, X, Y ) :        
        # no_of_training_examples, no_of_features        
        self.m, self.n = X.shape        
        # weight initialization        
        self.W = np.zeros( self.n )        
        self.b = 0        
        self.X = X        
        self.Y = Y
        # gradient descent learning
        for i in range( self.iterations ) :            
            self.update_weights()            
        return self
    # Helper function to update weights in gradient descent
    def update_weights( self ) :           
        A = 1 / ( 1 + np.exp( - ( self.X.dot( self.W ) + self.b ) ) )
        # calculate gradients        
        tmp = ( A - self.Y.T )        
        tmp = np.reshape( tmp, self.m )        
        dW = np.dot( self.X.T, tmp ) / self.m         
        db = np.sum( tmp ) / self.m 
        # update weights    
        self.W = self.W - self.learning_rate * dW    
        self.b = self.b - self.learning_rate * db
        return self
    # Hypothetical function  h( x ) 
    def predict( self, X ) :    
        Z = 1 / ( 1 + np.exp( - ( X.dot( self.W ) + self.b ) ) )        
        Y = np.where( Z > 0.5, 1, 0 )        
        return Y

In [104]:
tr_accuracy = list()
tr_precision = list()
te_accuracy = list()
te_precision = list()
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    standardize(X_train)
    standardize(X_test)
    obj1 = my_LogisticRegression(0.123,41000)
    model= obj1.fit(X_train,y_train)
    y_pred = obj1.predict(X_test)
    y_tr = obj1.predict(X_train)
    #Let's see the f1-score for training and testing data
    evaluation_tr_precision,evaluation_tr_accuracy = evaluation(y_tr,y_train)
    evaluation_te_precision,evaluation_te_accuracy= evaluation(y_test,y_pred)
    tr_precision.append(evaluation_tr_precision)
    tr_accuracy.append(evaluation_tr_accuracy)
    te_precision.append(evaluation_te_precision)
    te_accuracy.append(evaluation_te_accuracy)
print("Precision of train sets:",tr_precision)
print("Accuracy of train sets:",tr_accuracy)
print("Precision of test sets:",te_precision)
print("Accuracy of test sets:",te_accuracy)
print(np.mean(tr_precision)*100)
print(np.mean(tr_accuracy)*100)
print(np.mean(te_precision)*100)
print(np.mean(te_accuracy)*100)

Precision of train sets: [0.25, 0.4090909090909091, 0.2962962962962963]
Accuracy of train sets: [0.9757396449704142, 0.9822590183323477, 0.9751626256652868]
Precision of test sets: [0.4, 0.25, 0.2222222222222222]
Accuracy of test sets: [0.9692671394799054, 0.9562130177514793, 0.9715976331360947]
31.846240179573513
97.77204296560163
29.074074074074076
96.56925967891597


# CNN

In [105]:
tr_accuracy_cnn = list()
tr_loss_cnn = list()
te_accuracy_cnn = list()
te_loss_cnn = list()
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # determine the number of input features
    n_features = X_train.shape[1]
    # define model
    model = Sequential()
    model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
    model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # fit the model
    model.fit(X_train, y_train, epochs=10, batch_size=1500, verbose=0)
    # evaluate the model
    tr_loss,tr_acc = model.evaluate(X_train, y_train, verbose=0)
    te_loss,te_acc = model.evaluate(X_test, y_test, verbose=0)
    tr_loss_cnn.append(tr_loss)
    tr_accuracy_cnn.append(tr_acc)
    te_loss_cnn.append(te_loss)
    te_accuracy_cnn.append(te_acc)
print("Loss pf train sets:",tr_loss_cnn)
print("Accuracy of train sets:",tr_accuracy_cnn)
print("Loss pf test sets:",te_loss_cnn)
print("Accuracy of test sets:",te_accuracy_cnn)
print(np.mean(tr_loss_cnn)*100)
print(np.mean(tr_accuracy_cnn)*100)
print(np.mean(te_loss_cnn)*100)
print(np.mean(te_accuracy_cnn)*100)

Loss pf train sets: [3.0214431285858154, 8.85694408416748, 10.960881233215332]
Accuracy of train sets: [0.9715976119041443, 0.9739798903465271, 0.9680662155151367]
Loss pf test sets: [3.027498245239258, 11.654220581054688, 7.689098834991455]
Accuracy of test sets: [0.9704491496086121, 0.9656804800033569, 0.9775148034095764]
761.3089481989543
97.12145725886026
745.6939220428467
97.12148110071817
